<a href="https://colab.research.google.com/github/MarcoBessiNeo4j/Demos/blob/main/Aura_API_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---


# --------- Pre-requisites ---------


---


# Set credentials & parameters

**Retrieve credentials from the Aura Enterprise Console**

Go to Account settings / API keys / Generate API key

In [ ]:
from google.colab import userdata

In [ ]:
# change with your client Id on the Aura Enterprise console
clientId = userdata.get("CLIENT_ID")

# change with your client secret on the Aura Enterprise console
clientSecret = userdata.get("CLIENT_SECRET")

# change with the name of a region your Aura account is provisioned for
region = "europe-west3"  #'europe-west1'

print('clientId: '+clientId);
print('clientSecret: '+clientSecret);
print('region: '+region)

# Get API access token

In [ ]:
import requests

data = {
    'grant_type': 'client_credentials',
}

token_response = requests.post("https://api.neo4j.io/oauth/token", data=data, auth=(clientId, clientSecret))

print('HTTP '+str(token_response.status_code))
print(token_response.json())

In [ ]:
accessToken = token_response.json()["access_token"]
headers = {
    'Authorization': 'Bearer ' + accessToken,
    'Content-Type': 'application/json'
}

print('accessToken: '+accessToken)
print(headers)



---


# --------- Phase 1 ---------


---



# List instances

In [ ]:
response = requests.get('https://api.neo4j.io/v1beta5/instances', headers=headers)

print('HTTP '+str(response.status_code))
print(response.json())
for instance in response.json()["data"]:
  print(instance)

# Get tenant information

In [ ]:
idTenant='TO_PUT_HERE_ID_TENANT'
response = requests.get('https://api.neo4j.io/v1beta5/tenants/' + idTenant, headers=headers)

print('HTTP '+str(response.status_code))
print(response.json())

In [ ]:
# prompt: prettify json result

import json

# Assuming the last response.json() is the one you want to prettify
pretty_json_output = json.dumps(response.json(), indent=2)
pretty_json_output

# Create instance

In [ ]:
# https://neo4j.com/docs/aura/platform/api/specification/#/instances/post-instances
json_data = {
  "version": "5",
  "region": "europe-west1",
  "memory": "8GB",
  "storage": "16GB",
  "name": "Instance02",
  "type": "enterprise-ds",
  "tenant_id": "YOUR_PROJECT_ID",
  "cloud_provider": "gcp"
}

create_response = requests.post('https://api.neo4j.io/v1beta5/instances', headers=headers, json=json_data)

print('HTTP '+str(create_response.status_code))
print(create_response.json())

**Read useful values in response for later :**

In [ ]:
boltEndpoint = create_response.json()["data"]["connection_url"]
idAuraDb = create_response.json()["data"]["id"]
userName = create_response.json()["data"]["username"]
password = create_response.json()["data"]["password"]
print("boltEndpoint = " + boltEndpoint)
print("idAuraDb = " + idAuraDb);
print("userName = " + userName);
print("password = " + password);

# Get instance details

In [ ]:
idAuraDb='68049b7a'
response = requests.get('https://api.neo4j.io/v1beta5/instances/' + idAuraDb, headers=headers)

print('HTTP '+str(response.status_code))
print(response.json())

# Connect & add data

**Install a client app that can read & write to the database :**

In [ ]:
pip install neo4j

In [ ]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable

class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    def create_friendship(self, person1_name, person2_name):
        with self.driver.session(database="neo4j") as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.execute_write(
                self._create_and_return_friendship, person1_name, person2_name)
            for row in result:
                print("Created friendship between: {p1}, {p2}".format(p1=row['p1'], p2=row['p2']))

    @staticmethod
    def _create_and_return_friendship(tx, person1_name, person2_name):
        # To learn more about the Cypher syntax, see https://neo4j.com/docs/cypher-manual/current/
        # The Reference Card is also a good resource for keywords https://neo4j.com/docs/cypher-refcard/current/
        query = (
            "MERGE (p1:Person { name: $person1_name }) "
            "MERGE (p2:Person { name: $person2_name }) "
            "MERGE (p1)-[:KNOWS]->(p2) "
            "RETURN p1, p2"
        )
        result = tx.run(query, person1_name=person1_name, person2_name=person2_name)
        try:
            return [{"p1": row["p1"]["name"], "p2": row["p2"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    def find_person(self, person_name):
        with self.driver.session(database="neo4j") as session:
            result = session.execute_read(self._find_and_return_person, person_name)
            if len(result) ==0:
                print(f"No person found with name {person_name}")
            for row in result:
                print("Found person: {row}".format(row=row))

    @staticmethod
    def _find_and_return_person(tx, person_name):
        query = (
            "MATCH (p:Person) "
            "WHERE p.name = $person_name "
            "RETURN p.name AS name"
        )
        result = tx.run(query, person_name=person_name)
        return [row["name"] for row in result]

**Run the app (write data, and read to check it worked) :**

In [ ]:
    app = App(boltEndpoint, userName, password)
    app.create_friendship("Alice", "David")
    app.find_person("Alice")
    app.find_person("David")
    app.close()

# Pause instance

In [ ]:
response = requests.post('https://api.neo4j.io/v1beta5/instances/' + idAuraDb + '/pause', headers=headers)

print('HTTP '+str(response.status_code))
print(response.json())

**Check status :**

In [ ]:
response = requests.get('https://api.neo4j.io/v1beta5/instances/' + idAuraDb, headers=headers)

print('HTTP '+str(response.status_code))
print(response.json())

**Retry reading from neo4j database :**

In [ ]:
app = App(boltEndpoint, userName, password)
app.find_person("Alice")
app.close()

# Resume instance

In [ ]:
response = requests.post('https://api.neo4j.io/v1beta5/instances/' + idAuraDb + '/resume', headers=headers)

print('HTTP '+str(response.status_code))
print(response.json())

**Check status :**

In [ ]:
response = requests.get('https://api.neo4j.io/v1beta5/instances/' + idAuraDb, headers=headers)

print('HTTP '+str(response.status_code))
print(response.json())

**Retry reading from neo4j database :**

In [ ]:
app = App(boltEndpoint, userName, password)
app.find_person("Alice")
app.close()

# Delete instance

In [ ]:
response = requests.delete('https://api.neo4j.io/v1beta5/instances/' + idAuraDb, headers=headers)

print('HTTP '+str(response.status_code))
print(response.json())

**Check status :**


In [ ]:
response = requests.get('https://api.neo4j.io/v1beta5/instances/' + idAuraDb, headers=headers)

print('HTTP '+str(response.status_code))
print(response.json())



---


# --------- Phase 2 ---------


---



# Take snapshot

Pre-req : a running instance!

If you just deleted the instance, go back to "[Create instance](https://colab.research.google.com/drive/1ebpCW6AvS8dyhyhFSoUCucuo6fkyduCF#scrollTo=hBu7zD_7GJWm&line=1&uniqifier=1)"

In [ ]:
idAuraDb="68049b7a"
response = requests.post("https://api.neo4j.io/v1beta5/instances/"+idAuraDb+"/snapshots", headers=headers)

print('HTTP '+str(response.status_code))
print(response.json())

# List snapshots






In [ ]:
from datetime import date

date_filter=str(date.today()) # ISO8601 UTC date, ex: "2023-01-20"
ss_response = requests.get("https://api.neo4j.io/v1beta5/instances/"+idAuraDb+"/snapshots?date="+date_filter, headers=headers)

print('HTTP '+str(ss_response.status_code))
print(ss_response.json())
print("Snapshots:")
for snapshot in ss_response.json()["data"]:
  print(snapshot)

**Read id of last snapshot :**

In [ ]:
import urllib.parse

snapshots_sorted = sorted(ss_response.json()["data"], key=lambda s: s['timestamp'], reverse=True)
last_snapshot_id = snapshots_sorted[0]["snapshot_id"]

print("last_snapshot_id = " + last_snapshot_id);

# URL encoding should soon become unnecessary as snapshot_id format will change to uuid (ex :"ea027a70-fc7d-4094-874d-2b094ff0521f")
last_snapshot_id=urllib.parse.quote(last_snapshot_id) #'2023-01-20T11%3A03%3A27Z-adhoc'
print("last_snapshot_id (URL encoded) = " + last_snapshot_id);

# Restore snapshot

**Change data :**

In [ ]:
#change some data to prove backup/restore works : add "Bob"
app = App(boltEndpoint, userName, password)
app.create_friendship("Bob", "David")
app.find_person("Bob")
app.find_person("David")
app.close()

**Restore :**

In [ ]:
response = requests.post("https://api.neo4j.io/v1beta5/instances/"+idAuraDb+"/snapshots/"+last_snapshot_id+"/restore", headers=headers)

print('HTTP '+str(response.status_code))
print(response.json())

**Check status :**

In [ ]:
response = requests.get('https://api.neo4j.io/v1beta5/instances/' + idAuraDb, headers=headers)

print('HTTP '+str(response.status_code))
print(response.json())

**Check data :**

In [ ]:
#query to check that "Bob" is not there after restore
app = App(boltEndpoint, userName, password)
app.find_person("Alice")
app.find_person("David")
app.find_person("Bob")
app.close()



# Clone to new instance

Same endpoint as "create". Extra field 'source_instance_id' in inputs.

In [ ]:
payload = {
  "version": "5",
  "region": "europe-west1",
  "memory": "8GB",
  "storage": "16GB",
  "name": "Instance02",
  "type": "enterprise-ds",
  "source_instance_id": "db1d1234",
  "tenant_id": "YOUR_PROJECT_ID",
  "cloud_provider": "gcp"
}

clone_response = requests.post("https://api.neo4j.io/v1beta5/instances", headers=headers, json=payload)

print('HTTP '+str(clone_response.status_code))
print(clone_response.json())

**Read useful data in response for later :**

In [ ]:
idCloneDb = clone_response.json()["data"]["id"]
clone_boltEndpoint = clone_response.json()["data"]["connection_url"]
clone_userName = clone_response.json()["data"]["username"]
clone_password = clone_response.json()["data"]["password"]

print("idCloneDb = " + idCloneDb);
print("clone_boltEndpoint = " + clone_boltEndpoint)
print("clone_userName = " + clone_userName);
print("clone_password = " + clone_password);

**Check status :**

In [ ]:
response = requests.get('https://api.neo4j.io/v1beta5/instances/' + idCloneDb, headers=headers)

print('HTTP '+str(response.status_code))
print(response.json())

**Check data:**

In [ ]:
#query the data to verify it's present
app = App(clone_boltEndpoint, userName, password)
app.find_person("Alice")
app.find_person("David")
app.find_person("Bob")
app.close()

# Clone to existing instance (overwrite)

**Change data :**

In [ ]:
# change data on the clone, before overwriting it : add "Bobby"
app = App(clone_boltEndpoint, clone_userName, clone_password)
app.create_friendship("Bobby", "David")
app.find_person("Bobby")
app.close()

**Overwrite:**

In [ ]:
idSourceDb = '27dd2ab7' # idAuraDb
idTargetDb = '13483f5a' # idCloneDb
payload = {
  "source_instance_id": idSourceDb
}

response = requests.post( "https://api.neo4j.io/v1beta5/instances/"+idTargetDb+"/overwrite", headers=headers, json=payload)

print('HTTP '+str(response.status_code))
print(response.json())

**Check status :**

In [ ]:
response = requests.get('https://api.neo4j.io/v1beta5/instances/' + idTargetDb, headers=headers)

print('HTTP '+str(response.status_code))
print(response.json())

In [ ]:
# check whether "Bobby" is still there
app = App(clone_boltEndpoint, clone_userName, clone_password)
app.find_person("Bobby")
app.close()

# Rename & resize

In [ ]:
idModifiedDb='ec0ab5e0' #idAuraDb
payload = {
  "memory": '8GB',
  "name" : 'teamaura_renamed'
}

response = requests.request("PATCH", "https://api.neo4j.io/v1beta5/instances/"+idModifiedDb, headers=headers, json=payload)

print('HTTP '+str(response.status_code))
print(response.json())

**Check status :**

In [ ]:
response = requests.get('https://api.neo4j.io/v1beta5/instances/' + idModifiedDb, headers=headers)

print('HTTP '+str(response.status_code))
print(response.json())

# Tenants
A user can discover which regions, instance types, cloud providers, and sizes they are allowed to create databases in


## List tenants

In [ ]:
tenants_response = requests.get('https://api.neo4j.io/v1beta5/tenants', headers=headers)

print('HTTP '+str(tenants_response.status_code))
print(tenants_response.json())
tenant_id = tenants_response.json()["data"][0]["id"]
print(tenant_id)

## Instance Catalog

In [ ]:
tenant_response = requests.get('https://api.neo4j.io/v1beta5/tenants/'+tenant_id, headers=headers)

print('HTTP '+str(tenant_response.status_code))
print(tenant_response.json())
print("Instance configurations available: ")
for config in tenant_response.json()["data"]["instance_configurations"]:
  print(config)

# Cleanup

In [ ]:
cleanup_response = requests.delete('https://api.neo4j.io/v1beta5/instances/' + idAuraDb, headers=headers)

print('HTTP '+str(cleanup_response.status_code))
print(cleanup_response.json())

In [ ]:
cleanup_response = requests.delete('https://api.neo4j.io/v1beta5/instances/' + idCloneDb, headers=headers)

print('HTTP '+str(cleanup_response.status_code))
print(cleanup_response.json())